# Проект: Рекомендация тарифов

___
**Описание проекта**  

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».  
В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.
___
**Цель**  

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.
___
**Описание данных:**  

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:  

- `сalls` — количество звонков,
- `minutes` — суммарная длительность звонков в минутах,
- `messages` — количество sms-сообщений,
- `mb_used` — израсходованный интернет-трафик в Мб,
- `is_ultra` — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).
___
**План работы:**

1. Изучение общей информации
2. Предобработка данных
3. Разбивка данных на выборки
4. Исследование моделей
5. Проверка модели на тестовой выборке
6. Проверка модели на адекватность
7. Общий вывод

## Откроем файл с данными и изучим общую информацию

### Считаем данные из csv-файла в датафрейм и сохраним в переменную `df`. Путь к файлу:

`/datasets/users_behavior.csv`

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime
from scipy import stats as st

In [2]:
#Инициализация модели дерева решений
from sklearn.tree import DecisionTreeClassifier

#Инициализация модели случайного леса
from sklearn.ensemble import RandomForestClassifier

#Инициализация модели логистической регрессии
from sklearn.linear_model import LogisticRegression

#Поиск лучших параметров модели
from sklearn.model_selection import GridSearchCV

#Проверка модели на адекватность
from sklearn.dummy import DummyClassifier

#Разделение выборки в заданных пропорциях
from sklearn.model_selection import train_test_split

#Вычисление доли правильных ответов
from sklearn.metrics import accuracy_score

#Сохранение и загрузка обученной модели
import joblib

In [3]:
try:
    df = pd.read_csv('./users_behavior.csv')
except:
    print('File_not_found')

### Изучим общую информацию о полученном датафрейме

**Добавим настройки для удобства отображения данных**

In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)
pd.options.display.float_format = '{:,.2f}'.format

#Если необходимо вернуть дефолтные настройки, раскомментируй строку ниже
#pd.reset_option('all')

**Выведем первые 10 строчек датафрейма `df` на экран.**

In [5]:
df.head(10)

,calls,minutes,messages,mb_used,is_ultra
0,40.00,311.90,83.00,"19,915.42",0
1,85.00,516.75,56.00,"22,696.96",0
2,77.00,467.66,86.00,"21,060.45",0
3,106.00,745.53,81.00,"8,437.39",1
4,66.00,418.74,1.00,"14,502.75",0
5,58.00,344.56,21.00,"15,823.37",0
6,57.00,431.64,20.00,"3,738.90",1
7,15.00,132.40,6.00,"21,911.60",0
8,7.00,43.39,3.00,"2,538.67",1
9,90.00,665.41,38.00,"17,358.61",0


**Посмотрим на размер данных (количество строк, колонок).**

In [6]:
df.shape

(3214, 5)

**Просмотрим статистическую сводку каждой колонки, чтобы узнать распределение данных в каждой колонки. Используем метод `describe()`.**

In [7]:
df.describe(include = "all")

,calls,minutes,messages,mb_used,is_ultra
count,"3,214.00","3,214.00","3,214.00","3,214.00","3,214.00"
mean,63.04,438.21,38.28,"17,207.67",0.31
std,33.24,234.57,36.15,"7,570.97",0.46
min,0.00,0.00,0.00,0.00,0.00
25%,40.00,274.58,9.00,"12,491.90",0.00
50%,62.00,430.60,30.00,"16,943.24",0.00
75%,82.00,571.93,57.00,"21,424.70",1.00
max,244.00,"1,632.06",224.00,"49,745.73",1.00


**Выведем основную информацию о датафрейме с помощью метода `info()`.**

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


**Выведем количество пропущенных значений для каждой колонки.**

In [9]:
pd.DataFrame(df.isna().sum()).style.background_gradient('coolwarm')

,0
calls,0
minutes,0
messages,0
mb_used,0
is_ultra,0


**Выведем пропущенные значения в процентном соотношении.**

In [10]:
pd.DataFrame(df.isna().mean()).style.format("{:.2%}").background_gradient('coolwarm')

,0
calls,0.00%
minutes,0.00%
messages,0.00%
mb_used,0.00%
is_ultra,0.00%


**Проверим дубликаты.**

In [11]:
df.duplicated().sum()

0

**Выявлены следующие проблемы:**

>* тип значений `float64` в колонке `calls` следует изменить на `int64`
>* тип значений `float64` в колонке `messages` следует изменить на `int64`


## Предобработка данных

**Изменим тип значений `float64` в колонке `calls` на `int64`.**

In [12]:
df['calls'] = df['calls'].astype('int64')

**Изменим тип значений `float64` в колонке `messages` на `int64`.**

In [13]:
df['messages'] = df['messages'].astype('int64')

**В вашем распоряжении обучающий набор данных и целевой признак, который нужно предсказать по остальным признакам. Целевой признак — значение в колонке `is_ultra`. Такие задачи относятся к классу «обучение с учителем» (от англ. supervised learning).  
В нашем случае, целевой признак `категориальный`, тем самым будет решается задача `классификации`. Категорий всего две — речь идёт о `бинарной (двоичной) классификации`.**

**Многие библиотеки машинного обучения, для решения задач `классификации`, требуют, чтобы признаки были сохранены в отдельных переменных. Объявим две переменные:**  
* `features` (англ. «признаки») — запишем в неё колонки `calls`, `minutes`, `messages`, `mb_used`:
* `target` (англ. «цель») — целевой признак, запишем в неё колонку `is_ultra`.

In [14]:
features = df.drop(['is_ultra'], axis=1)
display(features.shape)

(3214, 4)

In [15]:
target = df['is_ultra']
display(target.shape)

(3214,)

## Разбъём данные на выборки

**Разделим исходные данные на `обучающую`, `валидационную` и `тестовую` выборки в пропорции `60`:`20`:`20`.**

**Данные запишем в новые переменные:**  

`features_train` - `60%`  
`features_valid` - `20%`  
`features_test` - `20%`  
`target_train` - `60%`  
`target_valid` - `20%`  
`target_test` - `20%`  

**В начале разделим данные на `обучающую` и `валидационную` в пропорции `60`:`40`.**

In [16]:
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.4, random_state=12345, stratify=target)

**Далее разделим данные на `валидационную` и `тестовую`, разделив полученную на предыдущем шаге `валидационную` выборку, в пропорции `50`:`50`, в итоге получим выборки в пропорции `60`:`20`:`20`.**

In [17]:
features_valid, features_test, target_valid, target_test = train_test_split(
    features_valid, target_valid, test_size=0.5, random_state=12345, stratify=target_valid)

**Посмотрим на полученные данные.**

In [18]:
display(features_train.shape, target_train.shape)

(1928, 4)

(1928,)

In [19]:
display(features_valid.shape, target_valid.shape)

(643, 4)

(643,)

In [20]:
display(features_test.shape, target_test.shape)

(643, 4)

(643,)

## Исследуем модели

**Для решения задач классификации, используем:**  

* модель `решающее дерево DecisionTreeClassifier`;
* модель `случайного леса RandomForestClassifier`;
* модель `логистической регрессии LogisticRegression`.

**Решающее дерево**

**Обучим модель `DecisionTreeClassifier` с используя цикл `for`.**

In [21]:
%%time

best_model = None
best_result = 0
for criterion in ['gini', 'entropy']:
    for max_depth in range(1, 11):
        for min_samples_split in range(2, 11):
            for min_samples_leaf in range(1, 11):
                model = DecisionTreeClassifier(criterion=criterion,
                                               max_depth=max_depth,
                                               min_samples_split=min_samples_split,
                                               min_samples_leaf=min_samples_leaf,
                                               random_state=12345)
                model.fit(features_train, target_train)
                result = model.score(features_valid, target_valid)
                if result > best_result:
                    best_model = model
                    best_result = result
        
print("Score лучшей модели:", best_result)
print("Гиперпараметры лучшей модели:", best_model)

Score лучшей модели: 0.8040435458786936
Гиперпараметры лучшей модели: DecisionTreeClassifier(max_depth=8, min_samples_leaf=3, min_samples_split=8,
                       random_state=12345)
CPU times: user 19.7 s, sys: 97.9 ms, total: 19.8 s
Wall time: 20 s


**Обучим модель `DecisionTreeClassifier` с помощью библиотеки `GridSearchCV`.**

In [22]:
model = DecisionTreeClassifier(random_state=12345)

param_search = {
    'criterion': ['gini', 'entropy'],
    'max_depth': list(range(1, 11)),
    'min_samples_split': list(range(2, 11)),
    'min_samples_leaf': list(range(1, 11))
}


gs = GridSearchCV(estimator=model,
                  cv=5,
                  param_grid=param_search,
                  scoring='accuracy')

**Для обучения и кросс-валидации объединим ранее разделенные `features_train`, `features_valid` и `target_train`, `target_valid` в новые переменные `features_for_gsearch` и `target_for_gsearch` соответственно.**

In [23]:
features_for_gsearch = pd.concat([features_train, features_valid])

In [24]:
target_for_gsearch = pd.concat([target_train, target_valid])

**Обучим модель на выборках `features_for_gsearch` и `target_for_gsearch`.**

In [25]:
%%time

gs.fit(features_for_gsearch, target_for_gsearch)

CPU times: user 2min 3s, sys: 338 ms, total: 2min 3s
Wall time: 2min 7s


GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=12345),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                         'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                         'min_samples_split': [2, 3, 4, 5, 6, 7, 8, 9, 10]},
             scoring='accuracy')

In [26]:
gs.best_estimator_

DecisionTreeClassifier(max_depth=7, min_samples_leaf=9, random_state=12345)

In [27]:
gs.best_score_

0.7942435117675947

**Сохраним обученную модель с помощью библиотеки `GridSearchCV` в `model_DecisionTreeClassifier.joblib`.**

In [28]:
joblib.dump(gs.best_estimator_, 'model_DecisionTreeClassifier.joblib')

['model_DecisionTreeClassifier.joblib']

**Случайный лес**

**Обучим модель `RandomForestClassifier` с используя цикл `for`.**

In [29]:
%%time

best_model = None
best_result = 0
for criterion in ['gini', 'entropy']:
    for n_estimators in (1, 101):
        for max_depth in range(2, 11):
            for min_samples_split in range(2, 11):
                for min_samples_leaf in range(1, 11):
                    model = RandomForestClassifier(n_jobs=-1,
                                                   criterion=criterion,
                                                   n_estimators=n_estimators,
                                                   max_depth=max_depth,
                                                   min_samples_split=min_samples_split,
                                                   min_samples_leaf=min_samples_leaf,
                                                   random_state=12345)
                    model.fit(features_train, target_train)
                    result = model.score(features_valid, target_valid)
                    if result > best_result:
                        best_model = model
                        best_result = result

display("Score наилучшей модели на валидационной выборке:", best_result)
display("Гиперпараметры лучшей модели:", best_model)

'Score наилучшей модели на валидационной выборке:'

0.8242612752721618

'Гиперпараметры лучшей модели:'

RandomForestClassifier(max_depth=10, min_samples_split=4, n_estimators=101,
                       n_jobs=-1, random_state=12345)

CPU times: user 18min 47s, sys: 1min 46s, total: 20min 33s
Wall time: 12min 21s


**Обучим модель `RandomForestClassifier` с помощью библиотеки `GridSearchCV`.**

In [30]:
model = RandomForestClassifier(n_jobs=-1, random_state=12345)

param_search = {
    'criterion': ['gini', 'entropy'],
    'n_estimators': list(range(1, 51)),
    'max_depth': list(range(1, 11)),
    'min_samples_split': list(range(2, 11))
}


gs = GridSearchCV(estimator=model,
                  cv=5,
                  param_grid=param_search,
                  scoring='accuracy')

In [31]:
%%time

gs.fit(features_for_gsearch, target_for_gsearch)

CPU times: user 47min 54s, sys: 4min 38s, total: 52min 32s
Wall time: 1h 12min 59s


GridSearchCV(cv=5,
             estimator=RandomForestClassifier(n_jobs=-1, random_state=12345),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                         'min_samples_split': [2, 3, 4, 5, 6, 7, 8, 9, 10],
                         'n_estimators': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                          13, 14, 15, 16, 17, 18, 19, 20, 21,
                                          22, 23, 24, 25, 26, 27, 28, 29, 30, ...]},
             scoring='accuracy')

In [32]:
gs.best_estimator_

RandomForestClassifier(criterion='entropy', max_depth=9, min_samples_split=10,
                       n_estimators=16, n_jobs=-1, random_state=12345)

In [33]:
gs.best_score_

0.8074662838578066

**Сохраним обученную модель с помощью библиотеки `GridSearchCV` в `model_RandomForestClassifier.joblib`.**

In [34]:
joblib.dump(gs.best_estimator_, 'model_RandomForestClassifier.joblib')

['model_RandomForestClassifier.joblib']

**Логистическая регрессия**

**Обучим модель `LogisticRegression` с используя цикл `for`.**

In [35]:
%%time

best_model = None
best_result = 0
for C in range(1, 1_001):
    model = LogisticRegression(max_iter=1_000, random_state=12345, C=C)
    model.fit(features_train, target_train)
    result = model.score(features_valid, target_valid)
    if result > best_result:
        best_model = model
        best_result = result

display("Score наилучшей модели на валидационной выборке:", best_result)
display("Гиперпараметры лучшей модели:", best_model)

'Score наилучшей модели на валидационной выборке:'

0.7387247278382582

'Гиперпараметры лучшей модели:'

LogisticRegression(C=1, max_iter=1000, random_state=12345)

CPU times: user 1min 20s, sys: 1.19 s, total: 1min 21s
Wall time: 40.7 s


**Обучим модель `LogisticRegression` с помощью библиотеки `GridSearchCV`.**

In [36]:
model = LogisticRegression(max_iter=1000, n_jobs=-1, random_state=12345)

param_search = {
    'C': list(range(1, 1_001))
}

gs = GridSearchCV(estimator=model,
                  cv=5,
                  param_grid=param_search,
                  scoring='accuracy')

In [37]:
%%time

gs.fit(features_for_gsearch, target_for_gsearch)

CPU times: user 53.7 s, sys: 1.71 s, total: 55.5 s
Wall time: 3min 7s


GridSearchCV(cv=5,
             estimator=LogisticRegression(max_iter=1000, n_jobs=-1,
                                          random_state=12345),
             param_grid={'C': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14,
                               15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26,
                               27, 28, 29, 30, ...]},
             scoring='accuracy')

In [38]:
gs.best_estimator_

LogisticRegression(C=2, max_iter=1000, n_jobs=-1, random_state=12345)

In [39]:
gs.best_score_

0.7355037588304183

**Сохраним обученную модель с помощью библиотеки `GridSearchCV` в `model_LogisticRegression.joblib`.**

In [40]:
joblib.dump(gs.best_estimator_, 'model_LogisticRegression.joblib')

['model_LogisticRegression.joblib']

#### Краткие выводы:

* модель `решающее дерево` показала `среднюю скорость обучения` и `средний результат`;
* модель `случайный лес` показала `низкую скорость обучения` и `высокий результат`;
* модель `логистическая регрессия` показала `более высокую скорость обучения` и `низкий результат`.

**Выберем модель `случайного леса RandomForestClassifier` как лучшую модель, и продолжим на ней тестирование.**

## Проверем модель на тестовой выборке

**Проверим модель `случайного леса RandomForestClassifier` на тестовой выборке.**

In [41]:
model = joblib.load('model_RandomForestClassifier.joblib')

In [42]:
model.score(features_test, target_test)

0.8289269051321928

## (бонус) Проверим модели на адекватность

In [43]:
strategies = ['most_frequent', 'prior', 'stratified', 'uniform']

for strategy in strategies:
    model = DummyClassifier(strategy=strategy, random_state=12345)
    model.fit(features_train, target_train)
    result = model.score(features_test, target_test)
    display(f'{strategy} = {result}')

'most_frequent = 0.6936236391912908'

'prior = 0.6936236391912908'

'stratified = 0.5567651632970451'

'uniform = 0.49455676516329705'

**Результаты `обученных моделей` на `тестовой выборке` показали результат выше, чем результаты `DummyClassifier`. Это даёт право сделать вывод, что модели проходят проверку на адекватность.**

## Общий вывод

**Были исследованы три модели:**

* Решающее дерево
* Случайный лес
* Логистическая регрессия

Наилучший результат показала модель `случайный лес`, качество которой, на `валидационной выборке`, показала результат - `0.807`, на `тестовой выборке` - `0.790`.


## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 3: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75
